In [6]:
import numpy as np

In [1]:
class SDES_cipher:
    def __init__(self, plainText, Key, IP, IPI, P10, P8, EXP, P4, S0, S1):
        self.plainText = plainText
        self.Key = Key
        self.IP = IP
        self.IPI = IPI
        self.P10 = P10
        self.P8 = P8
        self.EXP = EXP
        self.P4 = P4
        self.S0 = S0 
        self.S1 = S1
        
    def keyGen(self):
        K = []
        for idx in P10:
            K.append(Key[idx-1])
        shiftK1 = []
        shiftK2 = []
        for i in range(0,10):
            if i < 5:
                shiftK1.append(K[(i+1)%5])
            else:
                shiftK1.append(K[((i+1)%5)+5])
            if i < 5:
                shiftK2.append(K[(i+3)%5])
            else:
                shiftK2.append(K[((i+3)%5)+5])
        K1 = []
        K2 = []
        for idx in P8:
            K1.append(shiftK1[idx-1])
            K2.append(shiftK2[idx-1])
        return K1,K2
    
    def permText(self):
        PP = []
        for idx in IP:
            PP.append(plainText[idx-1])
        return PP
    
    def expand(self,right):
        RF = []
        for idx in EXP:   
            RF.append(right[idx-1])
        return RF

    def exor(self,right,K):
        for i in range(0,8):
            right[i] = right[i]^K[i]
        return right
    
    def funcF(self,l,r):
        L = []
        for i in range (0,4):
            L.append(l[i]^r[i])
        return L
    
    def Sboxes(self,bitss):
        idx = bitss[0]*2+bitss[3]*1
        jdx = bitss[1]*2+bitss[2]*1
        SB1 = S0[idx][jdx]
        idx = bitss[4]*2+bitss[7]*1
        jdx = bitss[5]*2+bitss[6]*1
        SB2 = S1[idx][jdx]
        SBL = []
        for item in SB1:
            SBL.append(item)
        for item in SB2:
            SBL.append(item)
        SBLL = []
        for idx in P4:
            SBLL.append(SBL[idx-1])
        return SBLL

In [10]:
plainText = [0,0,1,0,1,0,0,0]
Key = [1,1,0,0,0,1,1,1,1,0]
IP = [2,6,3,1,4,8,5,7]
IPI = [4,1,3,5,7,2,8,6]
P10 = [3,5,2,7,4,10,1,9,8,6]
P8 = [6,3,7,4,8,5,10,9]
EXP = [4,1,2,3,2,3,4,1]
P4 = [2,4,3,1]
S0 = [[(0,1),(0,0),(1,1),(1,0)],
      [(1,1),(1,0),(0,1),(0,0)],
      [(0,0),(1,0),(0,1),(1,1)],
      [(1,1),(0,1),(1,1),(1,0)]]
S1 = [[(0,0),(0,1),(1,0),(1,1)],
      [(1,0),(0,0),(0,1),(1,1)],
      [(1,1),(0,0),(0,1),(0,0)],
      [(1,0),(0,1),(0,0),(1,1)]]

cipher1 = SDES_cipher(plainText,Key,IP,IPI,P10,P8,EXP,P4,S0,S1)
K1, K2 = cipher1.keyGen()
permutedText = cipher1.permText()
print('Permuted Text is :', permutedText)

# Round 1
print()
print('----------ROUND 1---------')
L,R = permutedText[:4], permutedText[4:]
print('Left :', L)
print('Right :', R)

RE1 = cipher1.expand(R)
print('Expanded Right :', RE1)

RE1 = cipher1.exor(RE1,K1)
print('Right^Key :', RE1)

RE11 = cipher1.Sboxes(RE1)
print('After applying Sboxes and permuting result:', RE11)

LE11 = cipher1.funcF(L,RE11)
print('F() :', LE11)

# Round2
print()
print('----------ROUND 2---------')
L2,R2= R,LE11
print('Left :', L2)
print('Right :', R2)

RE2 = cipher1.expand(R2)
print('Expanded Right :', RE2)

RE2 = cipher1.exor(RE2,K2)
print('Right^Key :', RE2)

RE22 = cipher1.Sboxes(RE2)
print('After applying Sboxes and permuting result:', RE22)

LE22 = cipher1.funcF(L2,RE22)
print('F() :', LE22)

#Final merging
final = []
for item in LE22:
    final.append(item)
for item in R2:
    final.append(item)
    
cipherText = []
for idx in IPI:
    cipherText.append(final[idx-1])

print()
print('Cipher Text obtained is :',cipherText)

Permuted Text is : [0, 0, 1, 0, 0, 0, 1, 0]

----------ROUND 1---------
Left : [0, 0, 1, 0]
Right : [0, 0, 1, 0]
Expanded Right : [0, 0, 0, 1, 0, 1, 0, 0]
Right^Key : [1, 1, 1, 1, 1, 1, 0, 1]
After applying Sboxes and permuting result: [0, 0, 0, 1]
F() : [0, 0, 1, 1]

----------ROUND 2---------
Left : [0, 0, 1, 0]
Right : [0, 0, 1, 1]
Expanded Right : [1, 0, 0, 1, 0, 1, 1, 0]
Right^Key : [0, 0, 1, 1, 0, 0, 0, 1]
After applying Sboxes and permuting result: [0, 0, 1, 1]
F() : [0, 0, 0, 1]

Cipher Text obtained is : [1, 0, 0, 0, 1, 0, 1, 0]


In [11]:
# Decipher
print()
print('----------DECIPHERING---------')
plainText = cipherText
Key = [1,1,0,0,0,1,1,1,1,0]
IP = [2,6,3,1,4,8,5,7]
IPI = [4,1,3,5,7,2,8,6]
P10 = [3,5,2,7,4,10,1,9,8,6]
P8 = [6,3,7,4,8,5,10,9]
EXP = [4,1,2,3,2,3,4,1]
P4 = [2,4,3,1]
S0 = [[(0,1),(0,0),(1,1),(1,0)],
      [(1,1),(1,0),(0,1),(0,0)],
      [(0,0),(1,0),(0,1),(1,1)],
      [(1,1),(0,1),(1,1),(1,0)]]
S1 = [[(0,0),(0,1),(1,0),(1,1)],
      [(1,0),(0,0),(0,1),(1,1)],
      [(1,1),(0,0),(0,1),(0,0)],
      [(1,0),(0,1),(0,0),(1,1)]]

decipher = SDES_cipher(plainText,Key,IP,IPI,P10,P8,EXP,P4,S0,S1)
K2, K1 = decipher.keyGen()
permutedText = decipher.permText()
print('Permuted Text is :', permutedText)

# Round1
print()
print('----------ROUND 1---------')
L,R = permutedText[:4], permutedText[4:]
print('Left :', L)
print('Right :', R)

RE1 = decipher.expand(R)
print('Expanded Right :', RE1)

RE1 = decipher.exor(RE1,K1)
print('Right^Key :', RE1)

RE11 = decipher.Sboxes(RE1)
print('After applying Sboxes and permuting result:', RE11)

LE11 = decipher.funcF(L,RE11)
print('F() :', LE11)

# Round2
print()
print('----------ROUND 2---------')
L2,R2= R,LE11
print('Left :', L2)
print('Right :', R2)

RE2 = decipher.expand(R2)
print('Expanded Right :', RE2)

RE2 = decipher.exor(RE2,K2)
print('Right^Key :', RE2)

RE22 = decipher.Sboxes(RE2)
print('After applying Sboxes and permuting result:', RE22)

LE22 = decipher.funcF(L2,RE22)
print('F() :', LE22)

#Final merging
final = []
for item in LE22:
    final.append(item)
for item in R2:
    final.append(item)
    
decipherText = []
for idx in IPI:
    decipherText.append(final[idx-1])

print()
print('Decipher Text obtained is :',decipherText)


----------DECIPHERING---------
Permuted Text is : [0, 0, 0, 1, 0, 0, 1, 1]

----------ROUND 1---------
Left : [0, 0, 0, 1]
Right : [0, 0, 1, 1]
Expanded Right : [1, 0, 0, 1, 0, 1, 1, 0]
Right^Key : [0, 0, 1, 1, 0, 0, 0, 1]
After applying Sboxes and permuting result: [0, 0, 1, 1]
F() : [0, 0, 1, 0]

----------ROUND 2---------
Left : [0, 0, 1, 1]
Right : [0, 0, 1, 0]
Expanded Right : [0, 0, 0, 1, 0, 1, 0, 0]
Right^Key : [1, 1, 1, 1, 1, 1, 0, 1]
After applying Sboxes and permuting result: [0, 0, 0, 1]
F() : [0, 0, 1, 0]

Decipher Text obtained is : [0, 0, 1, 0, 1, 0, 0, 0]


In [3]:
text = 'sumitxyz'
plainText = []
for item in text:
    value = ord(item)-97)%26
    plainText.append("{0:008b}".format(value))

In [4]:
Key = [1,1,0,0,0,1,1,1,1,0]
IP = [2,6,3,1,4,8,5,7]
IPI = [4,1,3,5,7,2,8,6]
P10 = [3,5,2,7,4,10,1,9,8,6]
P8 = [6,3,7,4,8,5,10,9]
EXP = [4,1,2,3,2,3,4,1]
P4 = [2,4,3,1]
S0 = [[(0,1),(0,0),(1,1),(1,0)],
      [(1,1),(1,0),(0,1),(0,0)],
      [(0,0),(1,0),(0,1),(1,1)],
      [(1,1),(0,1),(1,1),(1,0)]]
S1 = [[(0,0),(0,1),(1,0),(1,1)],
      [(1,0),(0,0),(0,1),(1,1)],
      [(1,1),(0,0),(0,1),(0,0)],
      [(1,0),(0,1),(0,0),(1,1)]]

cipher2 = SDES_cipher(plainText,Key,IP,IPI,P10,P8,EXP,P4,S0,S1)
K1, K2 = cipher2.keyGen()
permutedText = cipher2.permText()
print('Permuted Text is :', permutedText)

# Round 1
print()
print('----------ROUND 1---------')
L,R = permutedText[:4], permutedText[4:]
print('Left :', L)
print('Right :', R)

RE1 = cipher2.expand(R)
print('Expanded Right :', RE1)

RE1 = cipher2.exor(RE1,K1)
print('Right^Key :', RE1)

RE11 = cipher2.Sboxes(RE1)
print('After applying Sboxes and permuting result:', RE11)

LE11 = cipher2.funcF(L,RE11)
print('F() :', LE11)

# Round2
print()
print('----------ROUND 2---------')
L2,R2= R,LE11
print('Left :', L2)
print('Right :', R2)

RE2 = cipher2.expand(R2)
print('Expanded Right :', RE2)

RE2 = cipher2.exor(RE2,K2)
print('Right^Key :', RE2)

RE22 = cipher2.Sboxes(RE2)
print('After applying Sboxes and permuting result:', RE22)

LE22 = cipher2.funcF(L2,RE22)
print('F() :', LE22)

#Final merging
final = []
for item in LE22:
    final.append(item)
for item in R2:
    final.append(item)
    
cipherText = []
for idx in IPI:
    cipherText.append(final[idx-1])

print()
print('Cipher Text obtained is :',cipherText)

Permuted Text is : [20, 23, 12, 18, 8, 25, 19, 24]

----------ROUND 1---------
Left : [20, 23, 12, 18]
Right : [8, 25, 19, 24]
Expanded Right : [24, 8, 25, 19, 25, 19, 24, 8]
Right^Key : [25, 9, 24, 19, 24, 19, 24, 9]


IndexError: list index out of range